<a href="https://colab.research.google.com/github/marcelogarret/projeto_mestrado/blob/main/Multivariate_Time_Series_Forecast_garret.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#DATA PREPARATION

In [17]:
# multivariate data preparation
from numpy import array
from numpy import hstack
# define input sequence
in_seq1 = array([10, 20, 30, 40, 50, 60, 70, 80, 90])
in_seq2 = array([15, 25, 35, 45, 55, 65, 75, 85, 95])
out_seq = array([in_seq1[i]+in_seq2[i] for i in range(len(in_seq1))])
# convert to [rows, columns] structure
in_seq1 = in_seq1.reshape((len(in_seq1), 1))
in_seq2 = in_seq2.reshape((len(in_seq2), 1))
out_seq = out_seq.reshape((len(out_seq), 1))
# horizontally stack columns
dataset = hstack((in_seq1, in_seq2, out_seq))
print(dataset)

[[ 10  15  25]
 [ 20  25  45]
 [ 30  35  65]
 [ 40  45  85]
 [ 50  55 105]
 [ 60  65 125]
 [ 70  75 145]
 [ 80  85 165]
 [ 90  95 185]]


In [18]:
# split a multivariate sequence into samples
def split_sequences(sequences, n_steps):
  X, y = list(), list()
  for i in range(len(sequences)):
    # find the end of this pattern
    end_ix = i + n_steps
    # check if we are beyond the dataset
    if end_ix > len(sequences):
      break
    # gather input and output parts of the pattern
    seq_x, seq_y = sequences[i:end_ix, :-1], sequences[end_ix-1, -1]
    X.append(seq_x)
    y.append(seq_y)
  return array(X), array(y)

In [19]:
# choose a number of time steps
n_steps = 3
# convert into input/output
X, y = split_sequences(dataset, n_steps)
print(X.shape, y.shape)

(7, 3, 2) (7,)


In [20]:
# summarize the data
for i in range(len(X)):
  print(X[i], y[i])

[[10 15]
 [20 25]
 [30 35]] 65
[[20 25]
 [30 35]
 [40 45]] 85
[[30 35]
 [40 45]
 [50 55]] 105
[[40 45]
 [50 55]
 [60 65]] 125
[[50 55]
 [60 65]
 [70 75]] 145
[[60 65]
 [70 75]
 [80 85]] 165
[[70 75]
 [80 85]
 [90 95]] 185


#MLP

In [21]:
# flatten input
n_input = X.shape[1] * X.shape[2]
X_mlp = X.reshape((X.shape[0], n_input))

# summarize the reshaped data
for i in range(len(X)):
  print(X_mlp[i], y[i])

[10 15 20 25 30 35] 65
[20 25 30 35 40 45] 85
[30 35 40 45 50 55] 105
[40 45 50 55 60 65] 125
[50 55 60 65 70 75] 145
[60 65 70 75 80 85] 165
[70 75 80 85 90 95] 185


In [22]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import Input

# define MLP model
model_mlp = Sequential()
model_mlp.add(Input(shape=(n_input,)))
model_mlp.add(Dense(100, activation='relu'))
model_mlp.add(Dense(1))
model_mlp.compile(optimizer='adam', loss='mse')

In [23]:
# fit model
model_mlp.fit(X_mlp, y, epochs=3000, verbose=0)

In [24]:
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_input))
yhat_mlp = model_mlp.predict(x_input, verbose=0)
print(yhat_mlp)

[[205.06137]]


# 1D CNN

In [25]:
n_features = X.shape[2]

In [26]:
from keras.layers import Flatten, Conv1D, MaxPooling1D

# define 1D CNN model
model_1dcnn = Sequential()
model_1dcnn.add(Input(shape=(n_steps, n_features)))
model_1dcnn.add(Conv1D(64, 2, activation='relu'))
model_1dcnn.add(MaxPooling1D())
model_1dcnn.add(Flatten())
model_1dcnn.add(Dense(50, activation='relu'))
model_1dcnn.add(Dense(1))
model_1dcnn.compile(optimizer='adam', loss='mse')

In [27]:
# fit model
model_1dcnn.fit(X, y, epochs=2000, verbose=0)

In [28]:
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_1dcnn = model_1dcnn.predict(x_input, verbose=0)
print(yhat_1dcnn)

[[206.10376]]


#LSTM

In [29]:
from keras.layers import LSTM

# define LSTM model
model_lstm = Sequential()
model_lstm.add(Input(shape=(n_steps, n_features)))
model_lstm.add(LSTM(50, activation='relu'))
model_lstm.add(Dense(1))
model_lstm.compile(optimizer='adam', loss='mse')

In [30]:
# fit model
model_lstm.fit(X, y, epochs=2000, verbose=0)

In [31]:
# demonstrate prediction
x_input = array([[80, 85], [90, 95], [100, 105]])
x_input = x_input.reshape((1, n_steps, n_features))
yhat_lstm = model_lstm.predict(x_input, verbose=0)
print(yhat_lstm)

[[205.48958]]


# RESULTS

In [32]:
print("MPL: " + f'{yhat_mlp[0][0]:0.2f}')
print("1D CNN: " + f'{yhat_1dcnn[0][0]:0.2f}')
print("LSTM: " + f'{yhat_lstm[0][0]:0.2f}')

MPL: 205.06
1D CNN: 206.10
LSTM: 205.49
